# Data generating module
Generate the synthetic data and then save using pickle (useful for importing to the other environment due to potential library conflicts).

In [1]:
import gpflux
import gpflow
import numpy as np
import tensorflow as tf
from gpflux.helpers import construct_basic_kernel, construct_basic_inducing_variables
from gpflux.layers import GPLayer
from gpflux.experiment_support.plotting import plot_layer
import pickle
from matplotlib import pyplot as plt
import ruptures as rpt
from scipy.stats import wasserstein_distance

2026-02-10 19:35:22.180717: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-10 19:35:24.490657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-10 19:35:25.232352: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-10 19:35:25.236569: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-10 19:35:26.515334: I tensorflow/core/platform/cpu_feature_gua

## Poisson Point Process

We start by defining an intensity function for the Non-Homogeneous Poisson process (NHPP) and then generate a Point process using a thinning algorithm that can be found here (https://www.columbia.edu/~ks20/4703-Sigman/4703-07-Notes-PP-NSPP.pdf)

In [2]:
def lam(x, a):
    """
    Intensity function for the NHPP
    """
    return a + np.sin(x)

In [3]:
def sample_poisson_jumps(a=10, lambda_star=11, length=1000, seed=None, segment_length=256):
    """
    Thinning algorithm for generating a poisson point process signal
    that alternates monotonicity every segment_length points.
    """
    if seed is not None:
        np.random.seed(seed)
    T = 0.01 * length
    t = 0.0
    n = 0
    arrival_times = []
    while True:
        u = np.random.rand()
        t = t - (1/lambda_star) * np.log(u)
        if t > T:
            break
        u = np.random.rand()
        lambda_t = lam(t, a)
        if u <= lambda_t / lambda_star:
            n += 2
            arrival_times.append(t)
    ts = np.linspace(0, T, length)
    ys = np.searchsorted(arrival_times, ts)
    

    increments = np.diff(ys, prepend=ys[0])
    
    segment_indices = np.arange(length) // segment_length
    signs = np.where(segment_indices % 2 == 0, 1, -1)
    ys_alternating = np.cumsum(increments * signs)
    
    return ts, ys_alternating, arrival_times, T

In [4]:
def sample_multiple_poisson(a, lambda_star, length, num_samples, seed=None):
    "Sample many Poisson base signals"
    signals = []
    tau = None
    tss = None
    for i in range(num_samples):
        current_seed = seed + i if seed is not None else None
        ts, ys, arrival_times, T = sample_poisson_jumps(a, lambda_star, length, current_seed)
        signals.append(ys)
        tau = T
        tss = ts
    return np.stack(signals, axis=0), tau, tss

## Deep Gaussian Process (DGP)

We continue with defining the hierarchical structure of the signals. We start by defining a helper function for sampling layer kernels and hyperparameters. The DGP model is constructed by first sampling from two stationary kernels supported by GPFlow for each layer while fixing the lengthscales and variance hyperparameters to 0.5 and 1.0 respectively. We then create a set of datapoints that are propagated through a number of predefined layers and save the output of the final layer.

In [5]:
# The kernels we will be using for the layers
KERNELS = [gpflow.kernels.SquaredExponential,
          gpflow.kernels.Matern12]

In [6]:
def randomly_sample_kernels(kernels, num_kernels, seed=None):
    """Helper function for sampling a corresponding kernel for each DGP layer."""
    if seed is not None:
        np.random.seed(seed)
    sampled_kernels = []
    for i in range(num_kernels):
        kernel = np.random.choice(kernels)()
        lengthscales = np.random.uniform(0.5, 0.5)
        variance = np.random.uniform(1.0, 1.0)
        kernel.lengthscales.assign(lengthscales)
        kernel.variance.assign(variance)
        sampled_kernels.append(kernel)
    return sampled_kernels

In [8]:
def sample_dgp_signal(segment_length, a, b, num_samples, kernels, num_layers, seed=None, reverse=False):
    """
    Sample num_samples trajectories where each segment of segment_length 
    corresponds to a DGP layer output.
    """
    if seed is not None:
        np.random.seed(seed)
        tf.random.set_seed(seed)
    
    X = np.linspace(a, b, segment_length).reshape(-1, 1)
    Z = X.copy()
    M = Z.shape[0]
    D = 1
    
    gp_layers = []
    for i in range(num_layers):
        ind_var = construct_basic_inducing_variables(M, D, D, share_variables=True, z_init=Z.copy())
        kernel = construct_basic_kernel(kernels[i], output_dim=D, share_hyperparams=True)
        layer_num_samples = num_samples if i == 0 else 1
        gplayer = GPLayer(
            kernel,
            ind_var,
            segment_length,
            full_cov=True,
            num_samples=layer_num_samples,
            mean_function=gpflow.mean_functions.Zero(),
        )
        gp_layers.append(gplayer)
    
    layer_input = X
    all_segments = []
    
    for i, layer in enumerate(gp_layers):
        layer_output = layer(layer_input)
        sample = tf.convert_to_tensor(layer_output)
        
        if i == 0:
            layer_input = sample
        else:
            layer_input = sample[0]
        
        segment = np.squeeze(layer_input.numpy(), axis=-1)
        all_segments.append(segment)
    
    # Reverse if requested: [L1, L2, L3, L4] → [L4, L3, L2, L1]
    if reverse:
        all_segments = all_segments[::-1]
    
    final = np.concatenate(all_segments, axis=1)
    
    total_length = segment_length * num_layers
    xs = np.linspace(a, b * num_layers, total_length)
    
    return final, all_segments, xs

In [9]:
def sequences(a, lambda_star, length, kernels, num_layers, num_samples,poisson_seed=None, kernel_seed=None, dgp_seed=None):
    segment_length = length // num_layers
    baselines, T, xs = sample_multiple_poisson(a, lambda_star, length, num_samples, poisson_seed)
    final, all_segments, xs = sample_dgp_signal(segment_length, 0, T, num_samples, kernels, num_layers, dgp_seed, reverse=True)
    seqs = baselines + final
    return T, baselines, seqs, all_segments

## We can now generate the time series we will need to train and evaluate our models on

We sample one synthetic dataset of num_samples = 1024 and seq_len = 1024. We produce the flat structure as described in the thesis.

In [10]:
dataset = {}

In [11]:
seed = 500
kernels = randomly_sample_kernels(KERNELS, 4, seed)
T, baselines, samples, segments = sequences(4, 5, 1024, kernels, 4, 1024, seed, seed, seed)


2026-01-27 22:48:52.674396: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
dataset["dgp_dataset"] = {"T": T, "baselines": baselines, "samples":samples, "segments":segments}

## We can now save the dataset in order to import it to another environment

In [13]:
from pathlib import Path

save_path = "data/synthetic_datasets_segments.pkl"
with open(save_path, "wb") as f:
    pickle.dump(dataset, f)
print(f"Saved to {save_path}")

Saved to data/synthetic_datasets_segments.pkl


# Estimation

After training and prediction, we are able to extract meaningful statistics out of the generated sequences which we can then use to quantitatively compare the outputs with the ground truth. First, we start with mapping each signal to a triple contatining changepoint count, jump mean differences and segment variances.

In [2]:
def estimate_cp(signal, pen=10):
    """Helper function which returns a ruptures model using the Pelt algorithm, an L2 segment cost function with a penalty (default 10)."""
    return rpt.Pelt(model='l2').fit(signal).predict(pen=pen)

In [49]:
def extract_signal_statistics(ys):
    # First, estimate the changepoints
    cps = estimate_cp(ys)
    cps = np.asarray(cps, dtype=int)
    
    # Get the boundaries of each changepoint
    boundaries = np.concatenate(([0], cps))
    inter_arrivals = np.diff(boundaries)
    num_cps = len(cps)

    means = []
    variances = []
    lengths = inter_arrivals
    segments = [(boundaries[i],boundaries[i+1]) for i in range(len(boundaries)-1)]
    for (start, end) in segments:
        segment = ys[start:end]
        means.append(segment.mean())
        variances.append(segment.var())
    # Local statistics, means, variances and lengths
    means = np.asarray(means)
    variances = np.asarray(variances)

    # Jumps in means
    jumps = np.diff(means)


    statistics =  {
        "cps" : cps,
        "num_cps" : num_cps,
        "inter_arrivals": inter_arrivals,
        "lengths": lengths,
        "means": means,
        "variances": variances,
        "jumps": jumps,
        }
    return statistics

## In-segment evaluation

In [50]:
with open("out/predictions/in_segment.pkl", "rb") as f:
    results = pickle.load(f)
with open("data/synthetic_datasets_segments.pkl", "rb") as f:
    samples = pickle.load(f)
signals = samples["dgp_dataset"]["samples"][819:]

In [51]:
seeds = [3040 + i for i in range(10)]
segments = [1, 2, 3, 4]

First, we evaluate the statistics of the ground truth signals.

In [52]:
gt_stats = {}
for seg_idx in segments:
    seg_start = (seg_idx - 1) * 256
    gt_stats[seg_idx] = []
    for j in range(signals.shape[0]): # go through all 205 signals.
        gt_signal = signals[j, seg_start + 128 : seg_start + 256]
        stats = extract_signal_statistics(gt_signal)
        gt_stats[seg_idx].append(stats)

Then, those of the predicted signals. These correspond to the 2nd half of the number of time points in the sequence.

In [53]:
pred_stats = {}
for seed in seeds:
    pred_stats[seed] = {}
    for seg_idx in segments:
        pred_stats[seed][seg_idx] = []
        for j in range(signals.shape[0]):
            # Get the 2nd half of the signal, that is the predicted section
            pred_signal = results[seed][f"segment{seg_idx}"][j, 128:]
            stats = extract_signal_statistics(pred_signal)
            pred_stats[seed][seg_idx].append(stats)

Now we can define the comparisons.

In [74]:
evaluation = {}
for seed in seeds:
    evaluation[seed] = {}
    for seg_idx in segments:
        # look through all segments with the same segment index
        gt_K = np.array([s["num_cps"] for s in gt_stats[seg_idx]])
        pred_K = np.array([s["num_cps"] for s in pred_stats[seed][seg_idx]])
        mae_K = np.mean(np.abs(gt_K - pred_K)) # 1) MAE
        gt_jumps = np.concatenate([s["jumps"] for s in gt_stats[seg_idx]])
        pred_jumps = np.concatenate([s["jumps"] for s in pred_stats[seed][seg_idx]])
        gt_vars = np.concatenate([s["variances"] for s in gt_stats[seg_idx]])
        pred_vars = np.concatenate([s["variances"] for s in pred_stats[seed][seg_idx]])
        w1_jumps = wasserstein_distance(gt_jumps, pred_jumps) # 2) W1(delta)
        w1_vars = wasserstein_distance(gt_vars, pred_vars) # 3) W1(sigma^2)
        evaluation[seed][seg_idx] = {
            "mae_K": mae_K,
            "w1_jumps": w1_jumps,
            "w1_vars": w1_vars,
        }

We use the metrics as defined in the thesis to get qualitative comparisons between ground truth and generated signals

In [75]:
for seg_idx in segments:
    mae_vals = [evaluation[seed][seg_idx]["mae_K"] for seed in seeds]
    w1j_vals = [evaluation[seed][seg_idx]["w1_jumps"] for seed in seeds]
    w1v_vals = [evaluation[seed][seg_idx]["w1_vars"] for seed in seeds]
    print(f"D={seg_idx} MAE:{np.mean(mae_vals):.4f} (+-) {np.std(mae_vals):.4f} | W1(delta): {np.mean(w1j_vals):.4f} (+-) {np.std(w1j_vals):.4f} | W1(sigma): {np.mean(w1v_vals):.4f} (+-) {np.std(w1v_vals):.4f}") 

D=1 MAE:1.5639 (+-) 0.1531 | W1(delta): 3.1508 (+-) 0.1380 | W1(sigma): 0.5371 (+-) 0.2831
D=2 MAE:1.6468 (+-) 0.2515 | W1(delta): 2.6030 (+-) 0.2039 | W1(sigma): 0.5860 (+-) 0.6593
D=3 MAE:1.8132 (+-) 0.1021 | W1(delta): 2.8115 (+-) 0.1461 | W1(sigma): 1.2780 (+-) 1.0447
D=4 MAE:1.8259 (+-) 0.1337 | W1(delta): 4.8902 (+-) 2.2672 | W1(sigma): 58.5480 (+-) 65.0997


Investigate a possible monotonic reversal trend.

In [78]:
for seg_idx in segments:
    gt_jumps = np.concatenate([s["jumps"] for s in gt_stats[seg_idx]])
    raw_w1 = []
    abs_w1 = []
    for seed in seeds:
        pred_jumps = np.concatenate([s["jumps"] for s in pred_stats[seed][seg_idx]])
        raw_w1.append(wasserstein_distance(gt_jumps, pred_jumps))
        abs_w1.append(wasserstein_distance(np.abs(gt_jumps), np.abs(pred_jumps)))
    print(f"D={seg_idx}  W1(Δ) raw={np.mean(raw_w1):.4f} (+-) {np.std(raw_w1):.4f} | W1(|Δ|) abs={np.mean(abs_w1):.4f} (+-) {np.std(abs_w1):.4f}")

D=1  W1(Δ) raw=3.1508 (+-) 0.1380 | W1(|Δ|) abs=0.2062 (+-) 0.0866
D=2  W1(Δ) raw=2.6030 (+-) 0.2039 | W1(|Δ|) abs=0.3972 (+-) 0.1784
D=3  W1(Δ) raw=2.8115 (+-) 0.1461 | W1(|Δ|) abs=0.3640 (+-) 0.1134
D=4  W1(Δ) raw=4.8902 (+-) 2.2672 | W1(|Δ|) abs=2.7341 (+-) 2.3553


Further evidence for a monotonic reversal, get the signs of the jumps in mean

In [85]:
for seg_idx in segments:
    gt_jumps = np.concatenate([s["jumps"] for s in gt_stats[seg_idx]])
    
    all_pred_means = []
    all_pred_stds = []
    for seed in seeds:
        pred_jumps = np.concatenate([s["jumps"] for s in pred_stats[seed][seg_idx]])
        all_pred_means.append(np.mean(pred_jumps))
        all_pred_stds.append(np.std(pred_jumps))
    print(f" D={seg_idx}: ")
    print(f"  Ground truth:   mean={np.mean(gt_jumps):.4f}, std={np.std(gt_jumps):.4f}")
    print(f"  Pred: mean={np.mean(all_pred_means):.4f} ± {np.std(all_pred_means):.4f}")

 D=1: 
  Ground truth:   mean=1.7913, std=0.6823
  Pred: mean=-1.3410 ± 0.1446
 D=2: 
  Ground truth:   mean=-1.4600, std=0.9659
  Pred: mean=1.0785 ± 0.1847
 D=3: 
  Ground truth:   mean=1.6733, std=1.1656
  Pred: mean=-1.0322 ± 0.1427
 D=4: 
  Ground truth:   mean=-1.4248, std=1.3257
  Pred: mean=1.0009 ± 0.3842


In [57]:
for seg_idx in segments:
    gt_jumps = np.concatenate([s["jumps"] for s in gt_stats[seg_idx] if len(s["jumps"]) > 0])
    gt_abs_max = np.abs(gt_jumps).max()
    
    raw_w1 = []
    abs_w1 = []
    clipped_abs_w1 = []
    outlier_pcts = []
    for seed in seeds:
        pred_jumps = np.concatenate([s["jumps"] for s in pred_stats[seed][seg_idx] if len(s["jumps"]) > 0])
        
        raw_w1.append(wasserstein_distance(gt_jumps, pred_jumps))
        abs_w1.append(wasserstein_distance(np.abs(gt_jumps), np.abs(pred_jumps)))
        
        # Clip absolute jumps to GT range
        pred_abs = np.abs(pred_jumps)
        outlier_pcts.append(100 * np.sum(pred_abs > gt_abs_max) / len(pred_abs))
        pred_clipped = np.clip(pred_abs, 0, gt_abs_max)
        clipped_abs_w1.append(wasserstein_distance(np.abs(gt_jumps), pred_clipped))
    
    print(f"D={5-seg_idx}: "
          f"W1(|Δ|)={np.mean(abs_w1):.4f} ± {np.std(abs_w1):.4f} | "
          f"W1(|Δ|) clipped={np.mean(clipped_abs_w1):.4f} ± {np.std(clipped_abs_w1):.4f} | "
          f"Outliers={np.mean(outlier_pcts):.1f}%")

D=4: W1(|Δ|)=0.2062 ± 0.0866 | W1(|Δ|) clipped=0.1910 ± 0.0819 | Outliers=1.3%
D=3: W1(|Δ|)=0.3972 ± 0.1784 | W1(|Δ|) clipped=0.3333 ± 0.1243 | Outliers=4.6%
D=2: W1(|Δ|)=0.3640 ± 0.1134 | W1(|Δ|) clipped=0.2364 ± 0.0532 | Outliers=3.7%
D=1: W1(|Δ|)=2.7341 ± 2.3553 | W1(|Δ|) clipped=0.3755 ± 0.1809 | Outliers=15.5%


In [86]:
for seg_idx in segments:
    gt_vars = np.concatenate([s["variances"] for s in gt_stats[seg_idx] if len(s["variances"]) > 0])
    
    all_pred_medians = []
    all_pred_means = []
    all_pred_stds = []
    for seed in seeds:
        pred_vars = np.concatenate([s["variances"] for s in pred_stats[seed][seg_idx] if len(s["variances"]) > 0])
        all_pred_medians.append(np.median(pred_vars))
        all_pred_means.append(np.mean(pred_vars))
        all_pred_stds.append(np.std(pred_vars))
    
    print(f"D={5-seg_idx}:")
    print(f"  GT:   median={np.median(gt_vars):.4f}, mean={np.mean(gt_vars):.4f}, "
          f"std={np.std(gt_vars):.4f}, range=[{gt_vars.min():.4f}, {gt_vars.max():.4f}]")
    print(f"  Pred: median={np.mean(all_pred_medians):.4f} ± {np.std(all_pred_medians):.4f}, "
          f"mean={np.mean(all_pred_means):.4f} ± {np.std(all_pred_means):.4f}, "
          f"std={np.mean(all_pred_stds):.4f} ± {np.std(all_pred_stds):.4f}")

D=4:
  GT:   median=0.6351, mean=0.7302, std=0.3927, range=[0.0828, 2.7961]
  Pred: median=0.5025 ± 0.1454, mean=1.0003 ± 0.4100, std=3.1500 ± 1.5932
D=3:
  GT:   median=0.5647, mean=0.6246, std=0.3409, range=[0.0126, 2.5544]
  Pred: median=0.5169 ± 0.1008, mean=1.0970 ± 0.7076, std=8.1770 ± 11.9512
D=2:
  GT:   median=0.3768, mean=0.4165, std=0.2483, range=[0.0083, 3.0486]
  Pred: median=0.2791 ± 0.0658, mean=1.5377 ± 1.0576, std=13.1492 ± 13.1270
D=1:
  GT:   median=0.2033, mean=0.2360, std=0.1682, range=[0.0010, 1.3212]
  Pred: median=0.2797 ± 0.0672, mean=58.7680 ± 65.0933, std=427.8630 ± 395.0301
